In [1]:
import numpy as np
import json
import geopandas as gpd
import pandas as pd
from keplergl import KeplerGl 

In [2]:
data_evolucion =pd.read_csv('covid19_tia_muni_y_distritos.csv',sep=";",encoding='ISO-8859-1')
data_evolucion['fecha_informe']=pd.to_datetime(data_evolucion['fecha_informe'])
data_evolucion['tasa_incidencia_acumulada_ultimos_14dias']=data_evolucion['tasa_incidencia_acumulada_ultimos_14dias'].str.replace(",",".")
data_evolucion['tasa_incidencia_acumulada_total']=data_evolucion['tasa_incidencia_acumulada_total'].str.replace(",",".")
data_evolucion.fillna(5,inplace=True)
casos_confirmados_ultimos_14dias=data_evolucion.groupby('codigo_geometria')['casos_confirmados_ultimos_14dias'].max()
tasa_incidencia_acumulada_ultimos_14dias=data_evolucion.groupby('codigo_geometria')['tasa_incidencia_acumulada_ultimos_14dias'].max()
data_evolucion.set_index('fecha_informe',inplace=True)

In [3]:
last_date=data_evolucion.index.max()
df_temp =data_evolucion[last_date:last_date]
#print(df_temp.columns)
with open('municipios_y_distritos_madrid.geojson','r',encoding='utf-8') as f:
    geojson = json.load(f)
    for feature in geojson['features']:
        featureProperties = feature['properties']
        codigo_geo = int(featureProperties['codigo_geo'])
        DSMUNI = featureProperties['DSMUNI']
        featureData = df_temp[df_temp['codigo_geometria'] == codigo_geo]
        featureProperties['casos_confirmados_totales'] =int(featureData.iloc[0]['casos_confirmados_totales'])
        featureProperties['tasa_incidencia_acumulada_total'] =float(featureData.iloc[0]['tasa_incidencia_acumulada_total'])
        featureProperties['max_casos_confirmados_14d'] =int(casos_confirmados_ultimos_14dias[codigo_geo])
        featureProperties['max_tasa_incidencia_14d'] =float(tasa_incidencia_acumulada_ultimos_14dias[codigo_geo])

    with open('latest.municipios_y_distritos_madrid.geojson','w',encoding='utf-8') as f:
        json.dump(geojson, f)

In [4]:
#df =pd.read_csv('../covid19_tia_muni_y_distritos (1).csv',sep=';',encoding='iso-8859-1') 
data_evolucion.reset_index(inplace=True)
gdf = gpd.read_file('municipios_y_distritos_madrid.geojson', encoding = 'utf-8')
data_evolucion['Lon']=np.tile(gdf.centroid.x,len(data_evolucion['fecha_informe'].unique()))
data_evolucion['Lat']=np.tile(gdf.centroid.y,len(data_evolucion['fecha_informe'].unique()))
data_evolucion.fillna(5,inplace=True)
data_evolucion.set_index(['fecha_informe','municipio_distrito'])
data_evolucion['casos_confirmados_totales_pct']=100*data_evolucion.sort_values('fecha_informe',ascending=True).groupby(['municipio_distrito'])['casos_confirmados_totales'].pct_change()
data_evolucion.to_csv('latest.covid19_tia_muni_y_distritos.csv',encoding='utf-8',index=False)

In [5]:
with open('latest.municipios_y_distritos_madrid.geojson','r',encoding='utf-8') as f:
    geojson = json.load(f)

In [6]:
map_1 = KeplerGl(height=1000)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [7]:
data_evolucion["fecha_informe"] = data_evolucion["fecha_informe"].astype(str)
map_1.add_data(data=data_evolucion, name="evolucion_14d")
map_1.add_data(data=geojson, name="total")


In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [9]:
with open('kepler.config') as json_file:
    kepler_config = json.load(json_file)
map_1.config =kepler_config

In [13]:
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'evolucion_14d', 'id': 'uvsxm…

In [11]:
kepler_config=map_1.config
with open("kepler.config","w",encoding='utf-8') as f:
    json.dump(kepler_config, f)

In [12]:
map_1.save_to_html(file_name=last_date.strftime("%Y%m%d") + '.coronavirus_madrid.html')

Map saved to 20200415.coronavirus_madrid.html!
